# Milestone 2 
------


## 1.  Tweets Acquisition
------
We had originally planned to use the twitter dataset that was given in the course. Unfortunatelly it was containing only 10% of the tweets in a given time period and wasn't including any information on the location of the user nor the user profile. Because of this we decided to go get the tweets about specific events by ourselves. 

### 1.1 Twitter API 
------
Our initial idea was to get the information we needed with the Twitter API, but there again we encountered several problems : 

- The **Rate Limit** of the Twitter API :  It would have taken a lot of time to get the tweets of a specific event, but we were ready to wait and launch the code on several computers.
- The **Search Query** limitations : After designing a code that would allow us to get the tweets by searching specific hashtags over a time interval, we discovered a huge limitation : tweets can only we searched with the API if they are *less than one week old*. 

So we have to discard the idea to use the Twitter API.

### 1.2 Scrapping Manually the Tweets 
------
Fortunatelly the twitter html interface (the website) allows us to search for any query on anytime interval. So we decided get the data by scraping directly the website. For that we use a browser that doesn't have a user interface **PhantomJS** and **Selenium** a python package that allows us to load urls in this browser and scroll down the search page in order to load results. Once loaded the use **Beautifull Soup 4** with the parser **LXML** To get every tweets of the page.

This was done using one script : [`tweet_acquisiton.py`](./TweetAcquisition/tweet_acquisition.py). 
For each event a new folder is created (for example here `Nigeria_1`), The query url is created using the list of hashtags specified inside the script. You can see the [`README.md`](./TweetAcquisition/README.md) file in the folder


## 2.  Geocoding the tweets
------

### 2.1 
------



## 3.  Enriching the Data
------

### 3.1 

------


## 4. Data Visualization


## 5. Critical Assessment

- Fact that twitter is biased by nature 
- Ideally we should have scrapped data from different social media
- Locations are never to be perfect, the location information is not objective. 
- Could have used Google API for example, but it is limited in the number of queries, and I won't be perfect either because usually google maps uses contextual infomation to find the location you are looking for. 


## 6. What's next ? 
